# Integration on level set domains
We describe how to use a level set description in ngsxfem. To illustrate this in the context of a simple example, in this notebook we want to calculate the area of the unit circle using the representation of it as integral over 1 over a implicitly defined subset of $[-1.5, 1.5]^2 \subset \mathbb{R}^2$.

Let $\phi$ be a continuous level set function. Then we define the implicitly given domains and the interface

$$
  \Omega_{-} := \{ \phi < 0 \}, \quad
  \Omega_{+} := \{ \phi > 0 \}, \quad
  \Gamma := \{ \phi = 0 \}.
$$

In [ ]:
import netgen.gui
%gui tk
import tkinter

# the constant pi
from math import pi
# ngsolve stuff
from ngsolve import *
# basic xfem functionality
from xfem import *
# basic geometry features (for the background mesh)
from netgen.geom2d import SplineGeometry
# visualization stuff
from ngsolve.internal import *

We generate the background mesh of the domain and use a simplicial triangulation

In [ ]:
square = SplineGeometry()
square.AddRectangle([-1.5,-1.5],[1.5,1.5],bc=1)
mesh = Mesh (square.GenerateMesh(maxh=0.8, quad_dominated=False))

On the background mesh we define the level set function. In this example we choose a levelset function which measures the distance to the origin and substracts a constant r, resulting in an circle of radius r as $\Omega_{-}$:

In [ ]:
r=1
levelset = sqrt(x*x+y*y)-r
Draw(levelset, mesh, "Levelset")
visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 0
visoptions.autoscale = 0

Using a levelset function like $\phi = \sqrt{x^2 + y^2} - 1$, we can represent the area of the unit circle as

$$
A = \pi = \int_{\Omega_-(\phi)} 1 \, \mathrm{d}x
$$

The ngsxfem function for integration on implicitly defined geometries is only able to handle functions $\phi$ which are in $\mathcal{P}^1(\Omega)$. Therefore we replace the levelset function $\phi$ with an approximation $\phi^\ast \in \mathcal{P}^1(\Omega)$. In order to do so, we can use the function InterpolateToP1(...,...):

In [ ]:
V = H1(mesh,order=1)
lset_approx = GridFunction(V)
InterpolateToP1(levelset,lset_approx)

Draw(lset_approx)
visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 0
visoptions.autoscale = 0

Notice that this replacement $\phi \mapsto \phi^\ast$ will introduce a geometry error of order $h^2$. We will investigate this numerically later on in this notebook.

As a next step we need to define an integrand. This should be a ngsolve CoefficientFunction. In order to measure the area of the unit circle, the CoefficientFunction should be the constant function with value 1:

In [ ]:
f = CoefficientFunction(1)

Now we are ready to call the Integrate Function. Additionally to the arguments known in NGSolve we need to pass the "levelset_domain" argument which has the following entries:
* "levelset": level set function which describes the geometry (best choice: P1-GridFunction)
* "domain_type": decision on which part of the geometry the integration should take place:
   * NEG : $\Omega_-^{lin}$
   * POS : $\Omega_+^{lin}$ 
   * IF : $\Gamma^{lin}$
* "force_intorder": fixed integration order (otherwise integration order is chosen depending on differential operators) - Note that integration does not imply geometrical accuracy.
* "subdivlvl": If subdivlvl > 1 is provided subdivision are applied. On these subdivision the "levelset" function is interpolated linearly and the integration is carried out. This option is deprecated. For higher order accuracy we refer to the isoparametric approach.

Further, we need (as regular NGSolve arguments) the integrand (f in our case), the mesh and the order which should be used for the underlying Gaussian quadrature rules. Here we use first a second order rule. Note that the geometry approximation is also only of low order. The call then looks as follows:

In [ ]:
order = 2
integral = Integrate(levelset_domain = { "levelset" : lset_approx, "domain_type" : NEG},
                             cf=f, mesh=mesh, order = order)
error = abs(integral - pi)
print("Result of the integration: ", integral)
print("Error of the integration: ", error)

In order to investigate the convergence behaviour of the sketched procedure, we now save the previous error and execute a refinement and repeat the steps from above to get a better approximation of $\pi$. You can repeadetly execute the lower block to have more refinement steps.

In [ ]:
errors = [error]

In [ ]:
# refine cut elements only:
RefineAtLevelSet(gf=lset_approx)

mesh.Refine()

V = H1(mesh,order=1)
lset_approx = GridFunction(V)
InterpolateToP1(levelset,lset_approx)

Draw(lset_approx)
visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 0
visoptions.autoscale = 0

integral = Integrate(levelset_domain = { "levelset" : lset_approx, "domain_type" : NEG},
                             cf=f, mesh=mesh, order = order)

error = abs(integral - pi)
print("Result of the integration: ", integral)
print("Error of the integration: ", error)

errors.append(error)
eoc = [log(errors[i+1]/errors[i])/log(0.5) for i in range(len(errors)-1)]

print("Collected L2 errors:", errors)
print("experimental order of convergence (L2):", eoc)

We observe second order convergence which result from the geometry approximation.
## Higher order geometry approximation with an isoparametric mapping
In order to get higher order convergence we can use the isoparametric mapping functionality of xfem.

We apply a mesh transformation technique in the spirit of isoparametric finite elements
[ [example (youtube)](https://youtu.be/Mst_LvfgPCg) ]:
![title](https://raw.githubusercontent.com/ngsxfem/ngsxfem/release/doc/graphics/lsetcurv.jpg)

To compute the corresponding mapping we use the LevelSetMeshAdaptation class

In [ ]:
# for isoparametric mapping
from xfem.lsetcurv import *
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=2)
deformation = lsetmeshadap.CalcDeformation(levelset)
Draw(deformation,mesh,"deformation")

We can observe the geometrical improvement in the following sequence:

In [ ]:
Draw(lsetmeshadap.lset_p1,mesh,"lsetp1")
Draw(deformation,mesh,"deformation")

visoptions.autoscale = 0
visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 1.0
visoptions.subdivisions = 4
from time import sleep

N=100000
deformation.vec[:] *= 1/N
for i in range (1,N+1):
    #sleep(0.1)
    deformation.vec[:] *= (i+1)/i
    Redraw()

To apply the transformation in the computation of the integrals we simply add 
* mesh.SetDeformation(deformation)
In the following script we compute the integrals using this technique on successively refined meshes.

In [ ]:
from xfem.lsetcurv import *

order = 4
refinements = 6
mesh = Mesh (square.GenerateMesh(maxh=0.8, quad_dominated=False))

levelset = sqrt(x*x+y*y)-1
referencevals = { POS : 9-pi, NEG : pi, IF : 2*pi }

lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order, threshold=0.2, discontinuous_qn=True)
lsetp1 = lsetmeshadap.lset_p1
errors_uncurved = dict()
errors_curved = dict()
eoc_uncurved = dict()
eoc_curved = dict()

for key in [NEG,POS,IF]:
    errors_curved[key] = []
    errors_uncurved[key] = []
    eoc_curved[key] = []
    eoc_uncurved[key] = []

f = CoefficientFunction (1.0)

for reflevel in range(refinements):
    if(reflevel > 0):
        mesh.Refine()

    for key in [NEG,POS,IF]:
        # Applying the mesh deformation
        deformation = lsetmeshadap.CalcDeformation(levelset)

        integrals_uncurved = Integrate(levelset_domain = { "levelset" : lsetp1, "domain_type" : key},
                                        cf=f, mesh=mesh, order = order)

        mesh.SetDeformation(deformation)
        integrals_curved = Integrate(levelset_domain = { "levelset" : lsetp1, "domain_type" : key},
                                       cf=f, mesh=mesh, order = order)
        # Unapply the mesh deformation (for refinement)
        mesh.UnsetDeformation()

        errors_curved[key].append(abs(integrals_curved - referencevals[key]))
        errors_uncurved[key].append(abs(integrals_uncurved - referencevals[key]))
    # refine cut elements:
    RefineAtLevelSet(gf=lsetmeshadap.lset_p1)

for key in [NEG,POS,IF]:
    eoc_curved[key] = [log(a/b)/log(2) for (a,b) in zip (errors_curved[key][0:-1],errors_curved[key][1:]) ]
    eoc_uncurved[key] = [log(a/b)/log(2) for (a,b) in zip (errors_uncurved[key][0:-1],errors_uncurved[key][1:]) ]

print("errors (  curved):  \n{}\n".format(  errors_curved))
print("   eoc (  curved):  \n{}\n".format(     eoc_curved))

print("avg.eoc(  curved):  \n{}\n".format(     sum(eoc_curved[IF][2:])/len(eoc_curved[IF][2:])))
print("avg.eoc(  curved):  \n{}\n".format(     sum(eoc_curved[NEG][2:])/len(eoc_curved[NEG][2:])))
print("avg.eoc(  curved):  \n{}\n".format(     sum(eoc_curved[POS][2:])/len(eoc_curved[POS][2:])))

You may try some other parameter values for (order, refinements) and compare the results.

We observe the higher order convergence that is desired. To see the application of this method for unfitted FEM, we refer to [unfittedFEM.ipynb](unfittedFEM.ipynb).